<a href="https://colab.research.google.com/github/KazGet/Pet-and-educational-projects/blob/main/%D0%9F%D1%80%D0%BE%D0%B4%D0%B0%D0%B6%D0%B8_%D0%B2_%D0%B3%D0%B8%D0%BF%D0%B5%D1%80%D0%BC%D0%B0%D1%80%D0%BA%D0%B5%D1%82%D0%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install lightgbm

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import lightgbm as lgb
from google.colab import drive
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
# Data processing
device = 'cuda' if torch.cuda.is_available() else 'cpu'

drive.mount('/content/drive')

# Загрузка данных
train_df = pd.read_csv('/content/drive/MyDrive/train1.csv')
test_df = pd.read_csv('/content/drive/MyDrive/test1.csv')

Mounted at /content/drive


In [ ]:
y = train_df['target']
X = train_df.drop(['target'], axis=1)

In [ ]:
X

,date,product_id,category_id,stock,sales,price,discount,cheques_last_sale,hours_last_sale
0,2024-05-08 01:00:00,1038040,780202,0.0,0.0,69.99,0.003600,71783.0,0.000000
1,2024-05-08 01:00:00,545978,740602,0.0,0.0,79.99,0.000000,71783.0,0.000000
2,2024-05-08 01:00:00,905362,680405,13.0,0.0,129.99,0.000000,626.0,9.285277
3,2024-05-08 01:00:00,545626,580605,11893.0,0.0,199.99,0.000000,933.0,53.618057
4,2024-05-08 01:00:00,545422,810402,9.0,0.0,1099.00,0.000000,67073.0,1228.055000
...,...,...,...,...,...,...,...,...,...
10537106,2024-07-31 23:00:00,932916,680705,0.0,0.0,94.99,0.002083,190185.0,695.000000
10537107,2024-07-31 23:00:00,934422,790205,0.0,0.0,0.00,0.000000,36164.0,599.000000
10537108,2024-07-31 23:00:00,934439,700303,0.0,0.0,239.00,0.000000,190185.0,695.000000
10537109,2024-07-31 23:00:00,934441,710202,48.0,0.0,64.99,0.000000,919.0,223.654720


In [ ]:
train_df['date'] = pd.to_datetime(train_df['date'])
test_df['date'] = pd.to_datetime(test_df['date'])

    # Извлечение временных признаков (примеры)
train_df['hour'] = train_df['date'].dt.hour
test_df['hour'] = test_df['date'].dt.hour
train_df['dayofweek'] = train_df['date'].dt.dayofweek  # Понедельник=0, Воскресенье=6
test_df['dayofweek'] = test_df['date'].dt.dayofweek

    # Кодирование категориальных признаков 'product' и 'category' с помощью Label Encoding
le_product = LabelEncoder()
train_df['product_encoded'] = le_product.fit_transform(train_df['product_id'])
test_df['product_encoded'] = le_product.transform(test_df['product_id'])

le_category = LabelEncoder()
train_df['category_encoded'] = le_category.fit_transform(train_df['category_id'])
test_df['category_encoded'] = le_category.transform(test_df['category_id'])

    # Выбор признаков для обучения
features = ['stock', 'sales', 'discount', 'price', 'cheques_last_sale',
                'hours_last_sale', 'hour', 'dayofweek', 'product_encoded',
                'category_encoded']
target = 'target'

X_train = train_df[features]
y_train = train_df[target]
X_test = test_df[features]

    # --- Шаг 3: Обучение модели ---

    # Выбор модели (в данном случае - Random Forest)
model = RandomForestClassifier(random_state=42)

In [ ]:
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [ ]:
predictions = model.predict(X_test)

In [ ]:
submission_df = pd.DataFrame({'id': test_df.index, 'target': predictions})
submission_df.to_csv('submission.csv', index=False)

print("Файл submission.csv с предсказаниями успешно создан.")

Файл submission.csv с предсказаниями успешно создан.


In [ ]:
templ = 0
for i in submission_df["target"]:
  if i==1:
    templ+=1
print(templ)

0


In [ ]:
X_train_split, X_val, y_train_split, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

    # Обучим модель на меньшей тренировочной выборке
model_eval = RandomForestClassifier(random_state=42)
model_eval.fit(X_train_split, y_train_split)

    # Сделаем предсказания на валидационной выборке
val_predictions = model_eval.predict(X_val)

    # Оценим качество модели
accuracy = accuracy_score(y_val, val_predictions)
print(f"\nТочность модели на валидационной выборке: {accuracy:.4f}")
print("\nОтчет о классификации на валидационной выборке:")
print(classification_report(y_val, val_predictions))


Точность модели на валидационной выборке: 0.9998

Отчет о классификации на валидационной выборке:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00   2106908
           1       0.00      0.00      0.00       343
           2       0.76      0.11      0.19       172

    accuracy                           1.00   2107423
   macro avg       0.59      0.37      0.40   2107423
weighted avg       1.00      1.00      1.00   2107423



In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from tensorflow import keras
from tensorflow.keras import layers

# --- Шаг 1: Загрузка данных (предполагается, что train_df и test_df уже загружены) ---
# Убедитесь, что вы выполнили код для загрузки данных из предыдущего примера

if 'train_df' not in locals() or 'test_df' not in locals():
    print("Данные не загружены. Пожалуйста, выполните шаг загрузки данных.")
else:
    # --- Шаг 2: Предварительная обработка данных ---

    # Преобразование столбца 'date' в datetime
    train_df['date'] = pd.to_datetime(train_df['date'])
    test_df['date'] = pd.to_datetime(test_df['date'])

    # Извлечение временных признаков
    train_df['hour'] = train_df['date'].dt.hour
    test_df['hour'] = test_df['date'].dt.hour
    train_df['dayofweek'] = train_df['date'].dt.dayofweek
    test_df['dayofweek'] = test_df['date'].dt.dayofweek
    train_df['month'] = train_df['date'].dt.month
    test_df['month'] = test_df['date'].dt.month

    # Кодирование категориальных признаков 'product' и 'category' с помощью One-Hot Encoding
    le_product = LabelEncoder()
    train_product_encoded = le_product.fit_transform(train_df['product_id'])
    test_product_encoded = le_product.transform(test_df['product_id'])

    le_category = LabelEncoder()
    train_category_encoded = le_category.fit_transform(train_df['category_id'])
    test_category_encoded = le_category.transform(test_df['category_id'])

    # Масштабирование числовых признаков
    numerical_features = ['stock', 'sales', 'discount', 'price', 'cheques_last_sale', 'hours_last_sale', 'hour', 'dayofweek', 'month']
    scaler = StandardScaler()
    train_numerical = scaler.fit_transform(train_df[numerical_features])
    test_numerical = scaler.transform(test_df[numerical_features])
    train_product_encoded = train_product_encoded.reshape(-1, 1)
    train_category_encoded = train_category_encoded.reshape(-1, 1)

    test_product_encoded = test_product_encoded.reshape(-1, 1)
    test_category_encoded = test_category_encoded.reshape(-1, 1)


In [ ]:
 # Объединение закодированных категориальных и масштабированных числовых признаков
X_train_arrays = np.concatenate([train_numerical, train_product_encoded, train_category_encoded], axis=1)
X_test_arrays = np.concatenate([test_numerical, test_product_encoded, test_category_encoded], axis=1)

    # Кодирование целевой переменной (если она еще не числовая)
label_encoder_target = LabelEncoder()
y_train_encoded = label_encoder_target.fit_transform(train_df['target'])

X_train_split, X_val, y_train_split, y_val = train_test_split(X_train_arrays, y_train_encoded, test_size=0.2, random_state=42, stratify=y_train_encoded)

# Определение количества классов
num_classes = len(np.unique(y_train_encoded))

# Определение входной формы
input_shape = X_train_split.shape[1]

# Создание простой модели
model = keras.Sequential([
    layers.Input(shape=(input_shape,)),
    layers.Dense(128, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(64, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])

# Компиляция модели
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',  # Используем sparse, так как метки целочисленные
              metrics=['accuracy'])

# Вывод сводки модели
model.summary()


epochs = 30
batch_size = 32

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 128)                 │           1,536 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 128)                 │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 64)                  │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 3)                   │             195 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 10,755 (42.01 KB)

 Trainable params: 10,371 (40.51 KB)

 Non-trainable params: 384 (1.50 KB)

In [ ]:

history = model.fit(X_train_split, y_train_split,
                        epochs=epochs,
                        batch_size=batch_size,
                        validation_data=(X_val, y_val),
                        verbose=1)  # verbose=1 показывает прогресс обучения



probabilities = model.predict(X_test_arrays)

predictions = np.argmax(probabilities, axis=-1)

original_predictions = label_encoder_target.inverse_transform(predictions)


submission_df = pd.DataFrame({'id': test_df.index, 'target': original_predictions})
submission_df.to_csv('submission.csv', index=False)

print("\nФайл submission.csv с предсказаниями успешно создан.")

import matplotlib.pyplot as plt

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Точность на обучающем наборе')
plt.plot(history.history['val_accuracy'], label='Точность на валидационном наборе')
plt.title('Точность модели')
plt.xlabel('Эпоха')
plt.ylabel('Точность')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Ошибка на обучающем наборе')
plt.plot(history.history['val_loss'], label='Ошибка на валидационном наборе')
plt.title('Ошибка модели')
plt.xlabel('Эпоха')
plt.ylabel('Ошибка')
plt.legend()

plt.tight_layout()
plt.show()

Epoch 1/30
263428/263428 ━━━━━━━━━━━━━━━━━━━━ 832s 3ms/step - accuracy: 0.9972 - loss: 0.0104 - val_accuracy: 0.9998 - val_loss: 0.0026
Epoch 2/30
 39783/263428 ━━━━━━━━━━━━━━━━━━━━ 9:50 3ms/step - accuracy: 0.9998 - loss: 0.0025